# Running custom model training on Vertex AI Pipelines

In this lab, you will learn how to run a custom model training job using the Kubeflow Pipelines SDK on Vertex AI Pipelines.

## Learning objectives

* Use the Kubeflow Pipelines SDK to build scalable ML pipelines.
* Create and containerize a custom Scikit-learn model training job that uses Vertex AI managed datasets.
* Run a batch prediction job within Vertex AI Pipelines.
* Use pre-built components, which are provided through the google_cloud_pipeline_components library, to interact with Vertex AI services.


## Vertex AI Pipelines setup
There are a few additional libraries you'll need to install in order to use Vertex AI Pipelines:

* __Kubeflow Pipelines__: This is the SDK you'll be using to build your pipeline. Vertex AI Pipelines supports running pipelines built with both Kubeflow Pipelines or TFX.
* __Google Cloud Pipeline Components__: This library provides pre-built components that make it easier to interact with Vertex AI services from your pipeline steps.

Install both of the services to be used in this notebook.

In [2]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!pip3 install -q --user 'kfp>=1.8.0,<2.0.0'
!pip3 install -q --user 'google-cloud-pipeline-components<2'

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


You may see some warning messages in the install output. It is safe to ignore those.

After installing these packages you'll need to restart the kernel:

In [3]:
# Automatically restart kernel after installs
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

Finally, check that you have correctly installed the packages. The KFP SDK version should be >=1.8:

In [1]:
!python3 -c "import kfp; print(f'KFP version: {kfp.__version__}')"
!python3 -c "import google_cloud_pipeline_components; print(f'google_cloud_pipeline_components version: {google_cloud_pipeline_components.__version__}')"

KFP version: 1.8.22
google_cloud_pipeline_components version: 1.0.45


### Import libraries
Add the following to import the libraries you'll be using throughout this notebook:

In [2]:
from datetime import datetime

from kfp.v2 import compiler, dsl

from google.cloud import aiplatform
from google_cloud_pipeline_components.aiplatform import TabularDatasetCreateOp, CustomContainerTrainingJobRunOp, ModelBatchPredictOp

import os

### Set your project ID and bucket
Throughout this notebook, you'll need your project ID. You'll also need a Cloud Storage bucket for the pipeline to work in. The following cell will automatically set your project ID and create your bucket.

This will also default your region to `us-central1`. Change this if you're running the Pipeline elsewhere.

In [3]:
PROJECT_ID = "qwiklabs-gcp-03-6da20d267875"
REGION = "us-central1"

# Get your Google Cloud project ID from gcloud
if not PROJECT_ID:
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

BUCKET_NAME=f"gs://{PROJECT_ID}-bucket"
print(BUCKET_NAME)
# make bucket if it doesn't exist
!gsutil ls $BUCKET_NAME > /dev/null || gsutil mb -l $REGION $BUCKET_NAME

#Grant service account access to the bucket.
shell_out = !gcloud auth list --format="value(account)"
SVC_ACCT = f'serviceAccount:{shell_out[0]}'
!gcloud storage buckets add-iam-policy-binding $BUCKET_NAME --member=$SVC_ACCT --role="roles/storage.admin"

gs://qwiklabs-gcp-03-6da20d267875-bucket
BucketNotFoundException: 404 gs://qwiklabs-gcp-03-6da20d267875-bucket bucket does not exist.
Creating gs://qwiklabs-gcp-03-6da20d267875-bucket/...
bindings:
- members:
  - serviceAccount:333587202436-compute@developer.gserviceaccount.com
  role: roles/storage.admin
- members:
  - projectEditor:qwiklabs-gcp-03-6da20d267875
  - projectOwner:qwiklabs-gcp-03-6da20d267875
  role: roles/storage.legacyBucketOwner
- members:
  - projectViewer:qwiklabs-gcp-03-6da20d267875
  role: roles/storage.legacyBucketReader
etag: CAI=
kind: storage#policy
resourceId: projects/_/buckets/qwiklabs-gcp-03-6da20d267875-bucket
version: 1


## Configuring a custom model training job
Before you set up your pipeline, you need to write the code for your custom model training job. To train the model, you'll use the UCI Machine Learning [Dry beans dataset](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset), from: KOKLU, M. and OZKAN, I.A., (2020), "Multiclass Classification of Dry Beans Using Computer Vision and Machine Learning Techniques."In Computers and Electronics in Agriculture, 174, 105507. [DOI](https://www.sciencedirect.com/science/article/abs/pii/S0168169919311573?via%3Dihub).

Your first pipeline step will create a managed dataset in Vertex AI using a BigQuery table that contains a version of this beans data. The dataset will be passed as input to your training job. In your training code, you'll have access to environment variable to access this managed dataset.

Here's how you'll set up your custom training job:

* Write a Scikit-learn `DecisionTreeClassifier` model to classify bean types in your data.
* Package the training code in a Docker container and push it to Container Registry

From there, you'll be able to start a Vertex AI Training job directly from your pipeline. Let's get started!

### Define your training code in a Docker container
Run the following to set up a directory where you'll add your containerized code:

In [4]:
!mkdir -p traincontainer/trainer
!touch traincontainer/Dockerfile
!touch traincontainer/trainer/train.py

After running those commands, you should see a directory called traincontainer/ created on the left (you may need to click the refresh icon to see it). You'll see the following in your traincontainer/ directory:

```
+ Dockerfile
+ trainer/
    + train.py
```
Your first step in containerizing your code is to create a Dockerfile. In your Dockerfile you'll include all the commands needed to run your image. It'll install all the libraries you're using and set up the entry point for your training code. Run the following to create a Dockerfile file locally in your notebook:

In [5]:
%%writefile traincontainer/Dockerfile
FROM us-docker.pkg.dev/vertex-ai/training/sklearn-cpu.1-0
WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

RUN pip install \
  scikit-learn==1.2 \
  'protobuf>=3.9.2,<3.20' \
  google-cloud-bigquery \
  google-cloud-storage \
  joblib \
  pandas \
  db_dtypes

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.train"]

Overwriting traincontainer/Dockerfile


Run the following to create `train.py` file. This retrieves the data from your managed dataset, puts it into a Pandas DataFrame, trains a Scikit-learn model, and uploads the trained model to Cloud Storage:

In [6]:
%%writefile traincontainer/trainer/train.py
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from google.cloud import storage
from joblib import dump

import os
import pandas as pd

bqclient = bigquery.Client()
storage_client = storage.Client()

def download_table(bq_table_uri: str):
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix):]

    table = bigquery.TableReference.from_string(bq_table_uri)
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe(create_bqstorage_client=False)

# These environment variables are from Vertex AI managed datasets
training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]

# Download data into Pandas DataFrames, split into train / test
df = download_table(training_data_uri)
test_df = download_table(test_data_uri)
labels = df.pop("Class").tolist()
data = df.values.tolist()
test_labels = test_df.pop("Class").tolist()
test_data = test_df.values.tolist()

skmodel = DecisionTreeClassifier()
skmodel.fit(data, labels)
score = skmodel.score(test_data, test_labels)
print('accuracy is:',score)

# Save the model to a local file
dump(skmodel, "model.joblib")

# Upload the saved model file to GCS
bucket = storage_client.get_bucket("YOUR_GCS_BUCKET")
model_directory = os.environ["AIP_MODEL_DIR"]
storage_path = os.path.join(model_directory, "model.joblib")
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename("model.joblib")

Overwriting traincontainer/trainer/train.py


Run the following to replace YOUR_GCS_BUCKET from the script above with the name of your Cloud Storage bucket:

In [7]:
BUCKET = BUCKET_NAME[5:] # Trim the 'gs://' before adding to train script
!sed -i -r "s/YOUR_GCS_BUCKET/$BUCKET/" traincontainer/trainer/train.py

You can also do this manually if you'd prefer. If you do, make sure not to include the gs:// in your bucket name when you update the script.

### Package training code as a container image

With your training code complete, you need to build a container image including your code and push it to Artifact Registry. Later when you configure the training component of your pipeline, you'll point Vertex AI Pipelines at this container image.

Create the Artifact Registry repository to hold our image.

In [8]:
REPO_NAME = "beans-model-trainer"
REPO_URI = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}"
IMAGE_NAME = "scikit-beans"
VERSION = "v1"

!gcloud artifacts repositories create $REPO_NAME --location $REGION --repository-format=docker

Create request issued for: [beans-model-trainer]
Waiting for operation [projects/qwiklabs-gcp-03-6da20d267875/locations/us-centr
al1/operations/3878b032-9e15-4e3d-b97c-310955215929] to complete...done.       
Created repository [beans-model-trainer].


Enable `gcloud` to help your docker client push images to Artifact Registry

In [9]:
!echo y | gcloud auth configure-docker $REGION-docker.pkg.dev


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
After update, the following will be written to your Docker config file located 
at [/home/jupyter/.docker/config.json]:
 {
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}

Do you want to continue (Y/n)?  
Docker configuration file updated.


Build your container by running the following cell. This will take a few minutes as it needs to download the base image.

In [10]:
!docker build ./traincontainer -t $REPO_URI/$IMAGE_NAME:$VERSION

Sending build context to Docker daemon   5.12kB
Step 1/5 : FROM us-docker.pkg.dev/vertex-ai/training/sklearn-cpu.1-0
latest: Pulling from vertex-ai/training/sklearn-cpu.1-0

aa388d02: Pulling fs layer 
d45cbc78: Pulling fs layer 
93dfdd76: Pulling fs layer 
8b7cea6b: Pulling fs layer 
8b7cea6b: Waiting fs layer 
5d8a1c31: Waiting fs layer 
815e9074: Pulling fs layer 
c53d84f1: Pulling fs layer 
c53d84f1: Waiting fs layer 
77ed9aa4: Pulling fs layer 
d373537f: Waiting fs layer 
77ed9aa4: Waiting fs layer 
6134ecfb: Pulling fs layer 
6134ecfb: Waiting fs layer 
90cd5c44: Pulling fs layer 
dfefd59a: Pulling fs layer 
8bca27c1: Pulling fs layer 
5ef74ae3: Waiting fs layer 
ca03e59f: Pulling fs layer 
e85b45eb: Pulling fs layer 
ae780ea1: Pulling fs layer 
9ac3c484: Pull complete 383kB/1.383kBBExtracting  11.14MB/818.4MBExtracting  215.6MB/261.9MBDigest: sha256:c1bc11aed8635650a6a1a1aebe7cabae3b2019c6e08f7aabf3d395b73442ba4f
Status: Downloaded newer image for us-docker.pkg.dev/vertex-ai/tra

Finally, push the container to Artifact Registry:

In [11]:
!docker push $REPO_URI/$IMAGE_NAME:$VERSION

The push refers to repository [us-central1-docker.pkg.dev/qwiklabs-gcp-03-6da20d267875/beans-model-trainer/scikit-beans]

77842a7a: Preparing 
ca42aab6: Preparing 
95c7b436: Preparing 
967c8575: Preparing 
57cbfd1c: Preparing 
95f9c6db: Preparing 
b11763b0: Preparing 
3a6f8e32: Preparing 
2d8c3713: Preparing 
5c4af0d6: Preparing 
425121a7: Preparing 
5005ccce: Preparing 
4140a22f: Preparing 
f57d328e: Preparing 
191ee4d5: Preparing 
492c2e8f: Preparing 
7f6e19c9: Preparing 
ad9fa2bc: Preparing 
17c0ab1b: Preparing 
daab5a5d: Preparing 
1ac55b04: Preparing 
1b153e96: Preparing 
7d9be9d9: Preparing 
492c2e8f: Pushed   2.189GB/2.14GBBPushing  291.1MB/1.215GBv1: digest: sha256:5e8630dd60264aa408e105491b0ba2b00b826cb4d85d9d3fdd83299b126071dd size: 5963


You can navigate to [Artifact Registry](https://console.cloud.google.com/artifacts) in the Console to verify your container is there.

## Configuring a batch prediction job
The last step of your pipeline will run a batch prediction job. For this to work, you need to provide a CSV file in Cloud Storage that contains the examples you want to get predictions on. You'll create this CSV file in your notebook and copy it to Cloud Storage using the `gsutil` command line tool.

### Copying batch prediction examples to Cloud Storage
The following file contains 3 examples from each class in your beans dataset. The example below doesn't include the `Class` column since that is what your model will be predicting. Run the following to create this CSV file locally in your notebook:

In [12]:
%%writefile batch_examples.csv
Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
23288,558.113,207.567738,143.085693,1.450653336,0.7244336162,23545,172.1952453,0.8045881703,0.9890847314,0.9395021523,0.8295857874,0.008913077034,0.002604069884,0.6882125787,0.9983578734
23689,575.638,205.9678003,146.7475015,1.403552348,0.7016945718,24018,173.6714472,0.7652721693,0.9863019402,0.8983750474,0.8431970773,0.00869465998,0.002711119968,0.7109813112,0.9978994889
23727,559.503,189.7993849,159.3717704,1.190922235,0.5430731512,24021,173.8106863,0.8037601626,0.9877607094,0.952462433,0.9157600082,0.007999299741,0.003470231343,0.8386163926,0.9987269085
31158,641.105,212.0669751,187.1929601,1.132879009,0.4699241567,31474,199.1773023,0.7813134733,0.989959967,0.9526231013,0.9392188582,0.0068061806,0.003267009878,0.8821320637,0.9993488983
32514,649.012,221.4454899,187.1344232,1.183349841,0.5346736437,32843,203.4652564,0.7849831,0.9899826447,0.9700068737,0.9188051492,0.00681077351,0.002994124691,0.8442029022,0.9989873701
33078,659.456,235.5600775,178.9312328,1.316483846,0.6503915309,33333,205.2223615,0.7877214708,0.9923499235,0.9558229607,0.8712102818,0.007121351881,0.002530662194,0.7590073551,0.9992209221
33680,683.09,256.203255,167.9334938,1.525623324,0.7552213942,34019,207.081404,0.80680321,0.9900349805,0.9070392732,0.8082699962,0.007606985006,0.002002710402,0.6533003868,0.9966903078
33954,716.75,277.3684803,156.3563259,1.773951126,0.825970469,34420,207.9220419,0.7994819873,0.9864613597,0.8305492781,0.7496238998,0.008168948587,0.001591181142,0.5619359911,0.996846984
36322,719.437,272.0582306,170.8914975,1.591993952,0.7780978465,36717,215.0502424,0.7718560075,0.9892420405,0.8818487005,0.7904566678,0.007490177594,0.001803782407,0.6248217437,0.9947124371
36675,742.917,285.8908964,166.8819538,1.713132487,0.8119506999,37613,216.0927123,0.7788277766,0.9750618137,0.8350248381,0.7558572692,0.0077952528,0.001569528272,0.5713202115,0.9787472145
37454,772.679,297.6274753,162.1493177,1.835514817,0.8385619338,38113,218.3756257,0.8016695205,0.9827093118,0.7883332637,0.7337213257,0.007946480356,0.001420623993,0.5383469838,0.9881438654
37789,766.378,313.5680678,154.3409867,2.031657789,0.8704771226,38251,219.3500608,0.7805870567,0.9879218844,0.8085170916,0.6995293312,0.008297866252,0.001225659709,0.4893412853,0.9941740339
47883,873.536,327.9986493,186.5201272,1.758516115,0.822571799,48753,246.9140116,0.7584464543,0.9821549443,0.7885506623,0.7527897207,0.006850002074,0.00135695419,0.5666923636,0.9965376533
49777,861.277,300.7570338,211.6168613,1.42123379,0.7105823885,50590,251.7499649,0.8019106536,0.9839296304,0.843243269,0.8370542883,0.00604208839,0.001829706116,0.7006598815,0.9958014989
49882,891.505,357.1890036,179.8346914,1.986207449,0.8640114945,51042,252.0153467,0.7260210171,0.9772736178,0.7886896753,0.7055518063,0.007160679276,0.001094585314,0.4978033513,0.9887407248
53249,919.923,325.3866286,208.9174205,1.557489212,0.7666552108,54195,260.3818974,0.6966846347,0.9825445152,0.7907120655,0.8002231025,0.00611066177,0.001545654241,0.6403570138,0.9973491406
61129,964.969,369.3481688,210.9473449,1.750902193,0.8208567513,61796,278.9836198,0.7501135067,0.9892064211,0.8249553283,0.7553404711,0.006042110436,0.001213219664,0.5705392272,0.9989583843
61918,960.372,353.1381442,224.0962377,1.575832543,0.7728529173,62627,280.7782864,0.7539207091,0.9886790043,0.8436218213,0.7950947556,0.005703319619,0.00140599258,0.6321756704,0.9962029945
141953,1402.05,524.2311633,346.3974998,1.513380332,0.7505863011,143704,425.1354762,0.7147107987,0.9878152313,0.9074598849,0.8109694843,0.003692991084,0.0009853172185,0.6576715044,0.9953071199
145285,1440.991,524.9567463,353.0769977,1.486805285,0.7400216694,146709,430.0960442,0.7860466375,0.9902937107,0.8792413513,0.8192980608,0.003613289371,0.001004269363,0.6712493125,0.9980170255
146153,1476.383,526.1933264,356.528288,1.475881001,0.7354662103,149267,431.3789276,0.7319360978,0.9791380546,0.8425962592,0.8198107159,0.003600290972,0.001003163512,0.6720896099,0.991924286

Writing batch_examples.csv


Then, copy the file to your Cloud Storage bucket:

In [13]:
!gsutil cp batch_examples.csv $BUCKET_NAME

Copying file://batch_examples.csv [Content-Type=text/csv]...
/ [1 files][  4.0 KiB/  4.0 KiB]                                                
Operation completed over 1 objects/4.0 KiB.                                      


You'll reference this file when you define your pipeline.

### Define constants
The last thing you need to do before building your pipeline is define some constant variables. `PIPELINE_ROOT` is the Cloud Storage path where the artifacts created by your pipeline will be written.

In [14]:
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

'gs://qwiklabs-gcp-03-6da20d267875-bucket/pipeline_root/'

After running the code above, you should see the root directory for your pipeline printed. This is the Cloud Storage location where the artifacts from your pipeline will be written. It will be in the format of `gs://YOUR-BUCKET-NAME/pipeline_root/`

### Building a pipeline with pre-built components
Now that your training code is packaged and available, you're ready to call it from your pipeline. The pipeline you'll define will use three pre-built components from the `google_cloud_pipeline_components` library you installed earlier. These predefined components simplify the code you need to write to set up your pipeline, and will allow us to use Vertex AI services like model training and batch prediction.

If you can't find a pre-built component for the task you want to accomplish, you can define your own Python-based custom components. To see an example, check out [this codelab](https://codelabs.developers.google.com/vertex-pipelines-intro#5).

Here's what your three-step pipeline will do:

* Create a managed dataset in Vertex AI.
* Run a training job on Vertex AI using the custom container you set up.
* Run a batch prediction job on your trained Scikit-learn classification model.

### Define your pipeline
Because you're using pre-built components, you can set up your entire pipeline in the pipeline definition.

In [15]:
@dsl.pipeline(name="automl-beans-custom",
              pipeline_root=PIPELINE_ROOT)
def my_pipeline(
    bq_source: str = "bq://specialized-training-resources.beans.beans1",
    bucket: str = BUCKET_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    bq_dest: str = "",
    container_uri: str = "",
    batch_destination: str = ""
):
    dataset_create_op = TabularDatasetCreateOp(
        display_name="tabular-beans-dataset",
        bq_source=bq_source,
        project=project,
        location=gcp_region
    )

    training_op = CustomContainerTrainingJobRunOp(
        display_name="pipeline-beans-custom-train",
        container_uri=container_uri,
        project=project,
        location=gcp_region,
        dataset=dataset_create_op.outputs["dataset"],
        staging_bucket=bucket,
        training_fraction_split=0.8,
        validation_fraction_split=0.1,
        test_fraction_split=0.1,
        bigquery_destination=bq_dest,
        model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
        model_display_name="scikit-beans-model",
        machine_type="e2-standard-4",
    )

    batch_predict_op = ModelBatchPredictOp(
        project=project,
        location=gcp_region,
        job_display_name="beans-batch-predict",
        model=training_op.outputs["model"],
        gcs_source_uris=[f"{BUCKET_NAME}/batch_examples.csv"],
        instances_format="csv",
        gcs_destination_output_uri_prefix=batch_destination,
        machine_type="n1-standard-2",
        starting_replica_count=1,
        max_replica_count=1
    )

### Compile and run the pipeline
With your pipeline defined, you're ready to compile it. The following will generate a JSON file that you'll use to run the pipeline:

In [16]:
compiler.Compiler().compile(
    pipeline_func=my_pipeline,
    package_path="custom_train_pipeline.json"
)

/home/jupyter/.local/lib/python3.9/site-packages/kfp/v2/compiler/compiler.py:1290: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


Next, define your pipeline job, passing in a few project-specific parameters:

In [17]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S") #job ID need to be unique

pipeline_job = aiplatform.PipelineJob(
    display_name="custom-train-pipeline",
    template_path="custom_train_pipeline.json",
    job_id=f"custom-train-pipeline-{TIMESTAMP}",
    parameter_values={
        "project": PROJECT_ID,
        "bucket": BUCKET_NAME,
        "bq_dest": f"bq://{PROJECT_ID}",
        "container_uri": f"{REPO_URI}/{IMAGE_NAME}:{VERSION}",
        "batch_destination": f"{BUCKET_NAME}/batch_results/"
    },
    enable_caching=True,
)

Finally, run the job to create a new pipeline execution:

In [18]:
pipeline_job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/333587202436/locations/us-central1/pipelineJobs/custom-train-pipeline-20240229133918
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/333587202436/locations/us-central1/pipelineJobs/custom-train-pipeline-20240229133918')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-train-pipeline-20240229133918?project=333587202436


After running this cell, you should see logs with a link to view the pipeline run in your console. Navigate to that link. You can also access it by opening your [Pipelines dashboard](https://console.cloud.google.com/vertex-ai/pipelines). This pipeline will take __25-35 minutes__ to run, but you can continue to the next step before it completes. Next you'll learn more about what's happening in each of these pipeline steps.

For the explanation and further instructions, please return to the lab page in Qwiklabs!